In [1]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [2]:
symbol = 'AAPL'
data = vbt.AlpacaData.download(symbol,
                               start='2023-3-05',
                               end='2023-3-16',
                               # limit=1000,
                               timeframe='15m').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


ConnectionError: HTTPSConnectionPool(host='data.alpaca.markets', port=443): Max retries exceeded with url: /v2/stocks/AAPL/bars?timeframe=15Min&adjustment=all&start=2023-03-05T05%3A00%3A00%2B00%3A00&end=2023-03-16T05%3A00%3A00%2B00%3A00&limit=500 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002174A1EF1F0>: Failed to establish a new connection: [WinError 10053] An established connection was aborted by the software in your host machine'))

In [ ]:
def is_hammer(close, high, low, start, ratio, girth):
    girth *= ratio
    fib = (high-low)*ratio
    lowerbound = high-fib

    is_hammer = np.where((start > lowerbound) & (((high-low)*girth) <= (close - start)), 1, 0)
    return is_hammer

    #if open > lowerbound and open-close > 0:
    #    return close.index[-1]
    #else:
    #    return None

IS_HAMMER = vbt.IndicatorFactory(
    class_name = "is_hammer",
    short_name = "is_hammer",
    input_names = ["close", "high", "low", "start"],
    output_names = ["value"],
    param_names=['ratio', 'girth'],
).from_apply_func(
    is_hammer
)

hammer = IS_HAMMER.run(data['Close'], data['High'], data['Low'], data['Open'], 0.32, .1)
data['is_hammer'] = hammer.value

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

#print(hammer.value)

#hammer = talib.CDLHAMMER(data['Open'], data['High'], data['Low'], #data['Close'])
#print(hammer[hammer==100])

In [ ]:
def hammer_buy(hammer_filter, high, close, n_shift = 3):

    #If 3 candles after hammer closes higher than hammer high
    high = ma.masked_array(high, mask=np.logical_not(hammer_filter), fill_value=np.inf).filled()
    high = np.roll(high, n_shift)
    high[:n_shift] = np.inf

    return np.where(close >= high, 1, 0)
HAMMER_BUY = vbt.IndicatorFactory(
    class_name = "hammer_buy",
    short_name = "hammer_buy",
    input_names = ['hammer', 'high', 'close',],
    output_names = ['buy_times'],
    param_names=['n_shift'],
).from_apply_func(hammer_buy, n_shift = 3)

buy_times = HAMMER_BUY.run(hammer.value, data['High'], data['Close'], n_shift=3)
data['buy'] = buy_times.buy_times
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

In [ ]:
def hammer_place_order(close, low, atr, risk_to_reward = 1.71, atr_multiplier = 1):

    #calculating buy, stop loss, take profit
    stop_loss = low - atr * atr_multiplier
    take_profit = close + (atr_multiplier * atr) * risk_to_reward

    return stop_loss, take_profit


HAMMER_PLACE_ORDER = vbt.IndicatorFactory(
    class_name = "hammer_place_order",
    short_name = "hammer_place_order",
    input_names = ['close', 'low', 'atr'],
    output_names = ['stop_loss', 'take_profit'],
    param_names=['risk_to_reward', 'atr_multiplier']
).from_apply_func(hammer_place_order, risk_to_reward = 1.71, atr_multiplier = 1)

ATR = talib.ATR(data['High'], data['Low'], data['Close'], 14)


order_prices = HAMMER_PLACE_ORDER.run(data['Close'], data['Low'], ATR, risk_to_reward = 1.71, atr_multiplier = 1)
data['stop_loss'] = order_prices.stop_loss
data['take_profit'] = order_prices.take_profit
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

